<a href="https://colab.research.google.com/github/BehnoodRasti/Unmixing_Tutorial_IEEE_IADF/blob/main/UnDIP_Colab_Apex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/BehnoodRasti/UnDIP

Cloning into 'UnDIP'...
remote: Enumerating objects: 173, done.
remote: Counting objects: 100% (173/173), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 173 (delta 71), reused 163 (delta 64), pack-reused 0
Receiving objects: 100% (173/173), 21.72 MiB | 39.79 MiB/s, done.
Resolving deltas: 100% (71/71), done.


In [5]:
# -*- coding: utf-8 -*-
"""
Created on Thu Apr  1 09:23:20 2021
@author: behnood
"""


from __future__ import print_function
import matplotlib.pyplot as plt
#%matplotlib inline

import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '3'

import numpy as np
from UnDIP.models import *

import torch
import torch.optim

from skimage.measure import compare_psnr
from skimage.measure import compare_mse
from utils.denoising_utils import *

from skimage._shared import *
from skimage.util import *
from skimage.metrics.simple_metrics import _as_floats
from skimage.metrics.simple_metrics import mean_squared_error

from UtilityMine import *

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark =True
dtype = torch.cuda.FloatTensor

PLOT = True
#%% Load image
import scipy.io
#%%
fname2  = "C:/Users/Behnood/Data/Samson/Y_clean.mat"
mat2 = scipy.io.loadmat(fname2)
img_np_gt = mat2["Y_clean"]
img_np_gt = img_np_gt.transpose(2,0,1)
[p1, nr1, nc1] = img_np_gt.shape
#%%
fname3  = "C:/Users/Behnood/Data/Samson/A_true.mat"
mat3 = scipy.io.loadmat(fname3)
A_true_np = mat3["A_true"]
A_true_np = A_true_np.transpose(2,0,1)
#%%
fname4  = "C:/Users/Behnood/Data/Samson/E.mat"
mat4 = scipy.io.loadmat(fname4)
E_np = mat4["E"]
n_lin=50
E_Lin=np.zeros((1,n_lin))
E_Lin[0,:]=np.linspace(0.5,1.5,n_lin)
rmax=E_np.shape[1]
EE=np.zeros((p1,rmax*n_lin))

#%%
import matplotlib.pyplot as plt
npar=np.zeros((1,4))
npar[0,0]=41
npar[0,1]=129
npar[0,2]=410
npar[0,3]=600
tol1=npar.shape[1]
tol2=5
Metric=np.zeros((tol2,10,tol1))
#mse_E=np.zeros((tol1,tol2))
save_result=False
import time
from tqdm import tqdm

ImportError: ignored